In [2]:
import os
import mido
from copy import deepcopy

In [ ]:
def shift_notes_and_save(midi_path: str, n: int, b: float):
    """
    Creates all permutations of a MIDI file by cyclically shifting the notes.

    Args:
        midi_path (str): Path to the MIDI file.
        n (int): Total number of beats in the MIDI file.
        b (float): Number of beats to shift by in each permutation.

    Returns:
        List[mido.MidiFile]: A list of MIDI files with shifted notes.
    """
    # Load the MIDI file
    original_midi = mido.MidiFile(midi_path)
    ticks_per_beat = original_midi.ticks_per_beat
    shift_ticks = int(ticks_per_beat * b)  # Convert beats to ticks for the shift

    # Calculate the number of permutations
    num_permutations = int(n / b)

    permutations = []
    for shift_index in range(num_permutations):
        # Deep copy the original MIDI to avoid altering it
        new_midi = deepcopy(original_midi)
        for track in new_midi.tracks:
            for msg in track:
                if not msg.is_meta and hasattr(msg, "time"):
                    # Adjust the time by the shift amount
                    msg.time = (msg.time + shift_ticks) % (ticks_per_beat * n)

        # Append the new MIDI file to the list
        permutations.append(new_midi)

    return permutations

In [ ]:
# Example usage
midi_permutations = shift_notes_and_save('path_to_midi_file.mid', 16, 1)
for i, midi_file in enumerate(midi_permutations):
    midi_file.save(f'output_shifted_{i}.mid')

In [5]:
rec_path = os.path.join("tmp", "recordings")

for recording in os.listdir(rec_path):
    print(f"playing {recording}")
    midi = mido.MidiFile(os.path.join(rec_path, recording))
    midi.print_tracks()

    with mido.open_output("Disklavier") as outport:
        for msg in midi.play():
            outport.send(msg)

playing recording-040-240501_143824.mid
=== Track 0
MetaMessage('set_tempo', tempo=1500000, time=0)
Message('note_on', channel=0, note=48, velocity=66, time=0)
Message('note_off', channel=0, note=48, velocity=53, time=72)
Message('note_on', channel=0, note=49, velocity=66, time=293)
Message('note_off', channel=0, note=49, velocity=52, time=79)
Message('note_on', channel=0, note=50, velocity=61, time=257)
Message('note_off', channel=0, note=50, velocity=54, time=101)
Message('note_on', channel=0, note=51, velocity=70, time=259)
Message('note_off', channel=0, note=51, velocity=51, time=65)
Message('note_on', channel=0, note=52, velocity=70, time=299)
Message('note_off', channel=0, note=52, velocity=56, time=103)
Message('note_on', channel=0, note=53, velocity=64, time=259)
Message('note_off', channel=0, note=53, velocity=53, time=79)
Message('note_on', channel=0, note=54, velocity=62, time=265)
Message('note_off', channel=0, note=54, velocity=58, time=67)
Message('note_on', channel=0, no

In [7]:
mbt = mido.bpm2tempo(1500000)
mspb = lambda tempo: int((60 / tempo) * 1_000_000)
print(mspb(40))

1500000
